In [13]:
from kamgon import *

In [8]:
class RobotNavigationProblem(Problem):
    """
    คลาสสำหรับปัญหาการนำทางของหุ่นยนต์ในโลกแบบตาราง (Grid World)

    คลาสนี้กำหนดโครงสร้างของปัญหาการค้นหาสำหรับหุ่นยนต์ที่เคลื่อนที่ในตารางที่มีสิ่งกีดขวาง
    โดยระบุการกระทำที่เป็นไปได้, ผลลัพธ์ของการกระทำเหล่านั้น, การทดสอบเป้าหมาย,
    ค่าใช้จ่ายของเส้นทาง และฟังก์ชันฮิวริสติกสำหรับอัลกอริทึมการค้นหาเช่น A*

    Attributes:
        grid (list[list[int]]): ตารางที่แสดงสภาพแวดล้อม โดย 0 คือเซลล์ที่เดินทางได้
                                 และ 1 คือสิ่งกีดขวาง
        rows (int): จำนวนแถวในตาราง
        cols (int): จำนวนคอลัมน์ในตาราง
    """
    def __init__(self, grid):
        """
        เริ่มต้นอ็อบเจกต์ RobotNavigationProblem

        Args:
            grid (list[list[int]]): ลิสต์ 2 มิติที่แสดงตาราง โดย 0 หมายถึงเส้นทางที่ไปได้
                                     และ 1 หมายถึงสิ่งกีดขวาง
        """
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])

    def actions(self, state):
        """
        กำหนดการกระทำที่เป็นไปได้จากสถานะที่กำหนด

        Args:
            state (tuple[int, int]): ตำแหน่งปัจจุบันของหุ่นยนต์ (แถว, คอลัมน์)

        Returns:
            list[str]: ลิสต์ของการกระทำที่ถูกต้อง ('UP', 'DOWN', 'LEFT', 'RIGHT')
        """
        possible_actions = []
        x, y = state
        # กำหนดการเคลื่อนที่ที่เป็นไปได้ 4 ทิศทาง (ขึ้น, ลง, ซ้าย, ขวา)
        moves = [
            (-1, 0, 'UP'),
            (1, 0, 'DOWN'),
            (0, -1, 'LEFT'),
            (0, 1, 'RIGHT')
        ]
        # ตรวจสอบการเคลื่อนที่แต่ละครั้งว่าเป็นไปได้หรือไม่
        for dx, dy, action in moves:
            new_x, new_y = x + dx, y + dy
            # การเคลื่อนที่จะถูกต้องถ้าอยู่ภายในขอบเขตของตารางและไม่ใช่สิ่งกีดขวาง
            if (0 <= new_x < self.rows and
                0 <= new_y < self.cols and
                self.grid[new_x][new_y] == 0):
                possible_actions.append(action)
        return possible_actions

    def result(self, state, action):
        """
        คำนวณสถานะผลลัพธ์จากการกระทำที่กำหนด

        Args:
            state (tuple[int, int]): สถานะปัจจุบัน (แถว, คอลัมน์)
            action (str): การกระทำที่กำลังทำ

        Returns:
            tuple[int, int]: สถานะใหม่หลังจากดำเนินการ
        """
        x, y = state
        # อัปเดตพิกัดตามการกระทำ
        if action == 'UP':
            return (x - 1, y)
        elif action == 'DOWN':
            return (x + 1, y)
        elif action == 'LEFT':
            return (x, y - 1)
        elif action == 'RIGHT':
            return (x, y + 1)
        # คืนค่าสถานะปัจจุบันหากการกระทำไม่ถูกต้อง (ซึ่งไม่ควรเกิดขึ้นหากใช้งานอย่างถูกต้อง)
        return state

    def goal_test(self, state, goal):
        """
        ตรวจสอบว่าสถานะปัจจุบันเป็นสถานะเป้าหมายหรือไม่

        Args:
            state (tuple[int, int]): สถานะปัจจุบัน
            goal (tuple[int, int]): สถานะเป้าหมาย

        Returns:
            bool: True ถ้าสถานะปัจจุบันคือเป้าหมาย, False ถ้าไม่ใช่
        """
        return state == goal

    def path_cost(self, c, state1, action, state2):
        """
        คำนวณค่าใช้จ่ายของเส้นทาง สำหรับกรณีอย่างง่ายนี้ แต่ละก้าวมีค่าใช้จ่ายเท่ากับ 1

        Args:
            c (float): ค่าใช้จ่ายที่สะสมมาจนถึง state1
            state1 (tuple[int, int]): สถานะเริ่มต้นของก้าว
            action (str): การกระทำที่ใช้เพื่อย้ายจาก state1 ไปยัง state2
            state2 (tuple[int, int]): สถานะผลลัพธ์

        Returns:
            float: ค่าใช้จ่ายทั้งหมดในการไปถึง state2
        """
        return c + 1

    def heuristic(self, state, goal):
        """
        คำนวณฮิวริสติกระยะทางแมนฮัตตันระหว่างสถานะปัจจุบันและเป้าหมาย

        ระยะทางแมนฮัตตันคือผลรวมของค่าสัมบูรณ์ของผลต่างของพิกัด x และ y
        ซึ่งเป็นฮิวริสติกที่ยอมรับได้ (admissible) สำหรับปัญหาแบบตาราง
        เพราะมันไม่เคยประเมินค่าใช้จ่ายจริงไปยังเป้าหมายสูงเกินไป

        Args:
            state (tuple[int, int]): สถานะปัจจุบัน (แถว, คอลัมน์)
            goal (tuple[int, int]): สถานะเป้าหมาย (แถว, คอลัมน์)

        Returns:
            int: ระยะทางแมนฮัตตันจากสถานะปัจจุบันไปยังเป้าหมาย
        """
        x1, y1 = state
        x2, y2 = goal
        return abs(x1 - x2) + abs(y1 - y2)


In [9]:
def visualize_path(grid, path, start, goal):
    """
    แสดงภาพตารางพร้อมเส้นทางที่คำนวณได้

    ฟังก์ชันนี้สร้างสำเนาของตารางและทำเครื่องหมายจุดเริ่มต้น ('S'),
    เป้าหมาย ('G'), และเส้นทาง ('*') เพื่อแสดงผล สิ่งกีดขวางจะแสดงเป็น '█'

    Args:
        grid (list[list[int]]): ตารางดั้งเดิม
        path (list[tuple[int, int]]): ลิสต์ของพิกัดในเส้นทาง
        start (tuple[int, int]): พิกัดเริ่มต้น
        goal (tuple[int, int]): พิกัดเป้าหมาย
    """
    # สร้างสำเนาของตาราง (deep copy) เพื่อหลีกเลี่ยงการแก้ไขตารางดั้งเดิม
    grid_copy = [row[:] for row in grid]

    # ทำเครื่องหมายเส้นทางบนตาราง โดยไม่รวมสถานะเริ่มต้นและเป้าหมาย
    for x, y in path:
        if (x, y) != start and (x, y) != goal:
            grid_copy[x][y] = '*'

    # ทำเครื่องหมายตำแหน่งเริ่มต้นและเป้าหมาย
    sx, sy = start
    gx, gy = goal
    grid_copy[sx][sy] = 'S'
    grid_copy[gx][gy] = 'G'

    # กำหนดสัญลักษณ์สำหรับการแสดงผล
    symbols = {0: ' ', 1: '█', '*': '•', 'S': 'S', 'G': 'G'}
    # พิมพ์แต่ละแถวของตารางโดยใช้สัญลักษณ์ที่กำหนด
    for row in grid_copy:
        print(''.join([symbols.get(cell, ' ') for cell in row]))

In [10]:
# --- ส่วนการทำงานหลัก ---
# กำหนดโครงสร้างของตาราง: 0 คือพื้นที่ว่าง, 1 คือสิ่งกีดขวาง
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

# กำหนดตำแหน่งเริ่มต้นและเป้าหมาย
initial = (0, 0)
goal = (4, 4)

print("--- กำลังทำงาน: การนำทางหุ่นยนต์แบบมาตรฐาน (A*) ---")
# สร้างอินสแตนซ์ของปัญหา
problem = RobotNavigationProblem(grid)
# ใช้อัลกอริทึม A* เพื่อค้นหาคำตอบ
solution_node = astar_search(problem, initial, goal)

# หากพบคำตอบ ให้พิมพ์เส้นทางและแสดงภาพ
if solution_node:
    path_coords = [node.state for node in solution_node.path()]
    print(f"พบเส้นทาง: {solution_node.solution()}")
    print(f"ค่าใช้จ่ายของเส้นทาง: {solution_node.path_cost}")
    visualize_path(grid, path_coords, initial, goal)
else:
    print("ไม่พบคำตอบ")

print("\n" + "="*40 + "\n")


--- กำลังทำงาน: การนำทางหุ่นยนต์แบบมาตรฐาน (A*) ---
พบเส้นทาง: ['DOWN', 'DOWN', 'DOWN', 'DOWN', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT']
ค่าใช้จ่ายของเส้นทาง: 8.0
S    
•██  
•    
•███ 
••••G


